In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from google.colab import files
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
uploaded = files.upload()

In [24]:
df = pd.read_csv("new_train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   age          32950 non-null  int64 
 1   job          32950 non-null  object
 2   marital      32950 non-null  object
 3   education    32950 non-null  object
 4   default      32950 non-null  object
 5   housing      32950 non-null  object
 6   loan         32950 non-null  object
 7   contact      32950 non-null  object
 8   month        32950 non-null  object
 9   day_of_week  32950 non-null  object
 10  duration     32950 non-null  int64 
 11  campaign     32950 non-null  int64 
 12  pdays        32950 non-null  int64 
 13  previous     32950 non-null  int64 
 14  poutcome     32950 non-null  object
 15  y            32950 non-null  object
dtypes: int64(5), object(11)
memory usage: 4.0+ MB


In [5]:
pd.DataFrame(df.apply(pd.unique)).rename(columns={0:'Unique values'})

,Unique values
age,"[49, 37, 78, 36, 59, 29, 26, 30, 50, 33, 44, 3..."
job,"[blue-collar, entrepreneur, retired, admin., s..."
marital,"[married, divorced, single, unknown]"
education,"[basic.9y, university.degree, basic.4y, high.s..."
default,"[unknown, no, yes]"
housing,"[no, yes, unknown]"
loan,"[no, yes, unknown]"
contact,"[cellular, telephone]"
month,"[nov, jul, may, jun, aug, mar, oct, apr, sep, ..."
day_of_week,"[wed, mon, tue, fri, thu]"


In [11]:
for i in ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'y']:
    print(df[i].value_counts())

admin.           8314
blue-collar      7441
technician       5400
services         3196
management       2345
retired          1366
entrepreneur     1160
self-employed    1099
housemaid         855
unemployed        798
student           711
unknown           265
Name: job, dtype: int64
married     19953
single       9257
divorced     3675
unknown        65
Name: marital, dtype: int64
university.degree      9736
high.school            7596
basic.9y               4826
professional.course    4192
basic.4y               3322
basic.6y               1865
unknown                1397
illiterate               16
Name: education, dtype: int64
no         26007
unknown     6940
yes            3
Name: default, dtype: int64
yes        17254
no         14900
unknown      796
Name: housing, dtype: int64
no         27131
yes         5023
unknown      796
Name: loan, dtype: int64
cellular     20908
telephone    12042
Name: contact, dtype: int64
no     29238
yes     3712
Name: y, dtype: int64


In [56]:

df1 = pd.concat([df[df['y'] == 'no'][0:4000], df[df['y'] == 'yes']])
for i in ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'y']:
    print(df1[i].value_counts())

admin.           2086
blue-collar      1447
technician       1228
services          653
management        579
retired           496
student           269
self-employed     259
entrepreneur      237
unemployed        218
housemaid         174
unknown            66
Name: job, dtype: int64
married     4401
single      2407
divorced     884
unknown       20
Name: marital, dtype: int64
university.degree      2498
high.school            1760
professional.course     976
basic.9y                962
basic.4y                737
basic.6y                402
unknown                 373
illiterate                4
Name: education, dtype: int64
no         6432
unknown    1280
Name: default, dtype: int64
yes        4115
no         3408
unknown     189
Name: housing, dtype: int64
no         6380
yes        1143
unknown     189
Name: loan, dtype: int64
cellular     5551
telephone    2161
Name: contact, dtype: int64
no     4000
yes    3712
Name: y, dtype: int64


In [57]:
df1 = df1.drop(axis=1, columns=['default', 'contact', 'month', 'day_of_week', 'campaign', 'previous', 'poutcome'])
df1.head()  

,age,job,marital,education,housing,loan,duration,pdays,y
0,49,blue-collar,married,basic.9y,no,no,227,999,no
1,37,entrepreneur,married,university.degree,no,no,202,999,no
3,36,admin.,married,university.degree,yes,no,120,999,no
4,59,retired,divorced,university.degree,no,no,368,999,no
5,29,admin.,single,university.degree,no,no,256,999,no


In [58]:
encoder = [preprocessing.LabelEncoder(), preprocessing.LabelEncoder(), preprocessing.LabelEncoder(), 
           preprocessing.LabelEncoder(), preprocessing.LabelEncoder(), preprocessing.LabelEncoder()]
for i, x in enumerate(['job', 'marital', 'education', 'housing', 'loan', 'y']): 
    encoder[i].fit(df1[x]) 
    df1[x] = encoder[i].transform(df1[x])
df1 

,age,job,marital,education,housing,loan,duration,pdays,y
0,49,1,1,2,0,0,227,999,0
1,37,2,1,6,0,0,202,999,0
3,36,0,1,6,2,0,120,999,0
4,59,5,0,6,0,0,368,999,0
5,29,0,2,6,0,0,256,999,0
...,...,...,...,...,...,...,...,...,...
32917,67,5,1,5,2,2,186,999,1
32918,21,8,2,3,0,0,400,999,1
32922,39,9,2,5,0,0,432,999,1
32937,29,9,2,6,2,2,437,999,1


In [59]:
y = df1['y']
x = df1.drop(columns=['y'])
scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [60]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [63]:
w0 = model.intercept_
w1 = model.coef_
w0
     

array([0.2363539])

In [65]:
w1

array([[ 0.27140189,  0.08713092,  0.28451181,  0.26105462,  0.06034525,
        -0.00744205,  1.8069476 , -1.01197004]])

In [67]:
print('y =', ' + '.join([ str(x) + "*x" + str(i)   for i, x in  enumerate(model.coef_[0])]), "+", model.intercept_[0] )

y = 0.2714018911514192*x0 + 0.08713091708767813*x1 + 0.28451181016356386*x2 + 0.2610546174739896*x3 + 0.06034524991111809*x4 + -0.007442050039042158*x5 + 1.8069475984871406*x6 + -1.0119700425314135*x7 + 0.23635390117418104


In [68]:
answers_pred = model.predict(x_test)

In [70]:
print(x_train.shape)
print(y_train.size)

(5398, 8)
5398


In [72]:
tn, fp, fn, tp = confusion_matrix(y_test, answers_pred).ravel()
print("accuracy:", (tp + tn)/(tp+fp+fn+tn))
rc = tp / (tp+fn)
pr = tp / (tp+fp)
print("recall:", rc)
print("precision:", pr)
print("f1:", 2*(rc*pr)/(rc+pr))

accuracy: 0.7882454624027658
recall: 0.7261682242990655
precision: 0.797741273100616
f1: 0.7602739726027397


In [73]:
print("accuracy:",accuracy_score(y_test, answers_pred))
print("recall:",recall_score(y_test, answers_pred))
print("precision:",precision_score(y_test, answers_pred))
print("f1:",f1_score(y_test, answers_pred))

accuracy: 0.7882454624027658
recall: 0.7261682242990655
precision: 0.797741273100616
f1: 0.7602739726027397
